In [1]:
import pandas as pd
import numpy as np

In [2]:
food = pd.read_csv("food.csv")
ratings = pd.read_csv("ratings.csv")
dataset = pd.read_csv("users_combined_rating_pivot.csv")
preds = pd.read_csv("predicted.csv")

In [3]:
# vv['originalid'] = uni.to_frame()
uni_ratings = ratings.User_ID.unique()
u_id = pd.DataFrame(pd.Series(uni_ratings))
u_id

,0
0,1.0
1,2.0
2,3.0
3,4.0
4,5.0
...,...
95,96.0
96,97.0
97,98.0
98,99.0


In [4]:
u_id[1] = u_id[0]
u_id.rename(columns = {0:'originalId', 1:'userId'}, inplace = True)
u_id

,originalId,userId
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0
3,4.0,4.0
4,5.0,5.0
...,...,...
95,96.0,96.0
96,97.0,97.0
97,98.0,98.0
98,99.0,99.0


In [5]:
u_id.to_csv('Id.csv', index=False)

In [6]:
uu = pd.read_csv("Id.csv")
uu.head()

,originalId,userId
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0
3,4.0,4.0
4,5.0,5.0


In [7]:
uu = uu.astype({"originalId": str})

In [8]:
fil = uu['originalId'].isin(['2.0'])
fil2 = fil.any()
if(fil2):
    uid = uu.loc[fil]['userId']
    print(str(float(uid)))

2.0


In [9]:
last = float(uu.tail(1).userId)
uu.loc[len(uu.index)] = ["2345", last+1]
uu.tail()

,originalId,userId
96,97.0,97.0
97,98.0,98.0
98,99.0,99.0
99,100.0,100.0
100,2345,101.0


In [10]:
import json

# some JSON:
x =  '{ "userID":"2345", "Food_Name": ["tricolour salad","banana walnut smoothie","andhra pan fried pomfret"]}'

# parse x:
order_data = json.loads(x)
User_id = order_data["userID"]
del order_data["userID"]
order_data = pd.DataFrame(order_data)
order_data

,Food_Name
0,tricolour salad
1,banana walnut smoothie
2,andhra pan fried pomfret


In [11]:
def svd_new_user_order(User_id, oder_data):
    ratings2 = pd.read_csv("ratings.csv")
#     ratings2.drop(ratings2.tail(1).index,inplace=True)
    food = pd.read_csv("food.csv")
    uu = pd.read_csv("Id.csv")
    uu = uu.astype({"originalId": str})
    fil = uu['originalId'].isin([User_id])
    fil2 = fil.any()
    if(fil2):
        uid = uu.loc[fil]['userId']
        uid = str(float(uid))
    else:
        uid = str(float(uu.tail(1).userId)+1)
        uu.loc[len(uu.index)] = [User_id, uid]
        uu.to_csv("Id.csv", index=False)
    for ind in oder_data.index:
#         print(rating_data_nested_list['Food_ID'][ind], rating_data_nested_list['rating'][ind])
        foodName = order_data["Food_Name"][ind]
        flt = food['Name'] == foodName
        foodId = int(food.loc[flt]['Food_ID'])
        flt = ratings2['Food_ID'] == foodId
        food_rating = ratings2.loc[flt]['Rating'].mean()
        ratings2.loc[len(ratings2.index)] = [uid, foodId, food_rating]
    
    ratings2.to_csv('ratings2.csv', index=False)
    dataset_2 = ratings2.pivot_table(index='User_ID',columns='Food_ID',values='Rating')
    dataset_2.fillna(0,inplace=True)
    R = dataset_2.values
    user_ratings_mean = np.mean(R, axis = 1)
    Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)
    
    from scipy.sparse.linalg import svds
    U, sigma, Vt = svds(Ratings_demeaned, k = 50)
    sigma = np.diag(sigma)
    all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
    preds2 = pd.DataFrame(all_user_predicted_ratings, columns = dataset_2.columns)
    preds2.to_csv('predicted2.csv')
    return

In [12]:
svd_new_user_order(User_id, order_data)

In [13]:
# import json

# # some JSON:
# j =  '{ "userID":"1234", "Food_rating": [{"Food_ID": "abc","rating": "5"},{"Food_ID": "bcd","rating": "6"},{"Food_ID": "cde","rating": "7"}]}'

# # parse x:
# rating_data = json.loads(j)
# User_id = rating_data["userID"]
# rating_data_nested_list = pd.json_normalize(rating_data, record_path =['Food_rating'])
# rating_data_nested_list

In [14]:
# rating_data_nested_list["rating"] = rating_data_nested_list["rating"].map(float)

In [15]:
# def svd_new_user_rating(User_id, rating_data_nested_list):
#     ratings2 = pd.read_csv("ratings.csv")
#     ratings2.drop(ratings2.tail(1).index,inplace=True)
#     for ind in rating_data_nested_list.index:
# #         print(rating_data_nested_list['Food_ID'][ind], rating_data_nested_list['rating'][ind])
#         ratings2.loc[len(ratings2.index)] = [User_id, rating_data_nested_list['Food_ID'][ind], rating_data_nested_list['rating'][ind]]
    
#     preds.to_csv('ratings2.csv')
#     dataset_2 = ratings2.pivot_table(index='User_ID',columns='Food_ID',values='Rating')
#     dataset_2.fillna(0,inplace=True)
#     R = dataset_2.values
#     user_ratings_mean = np.mean(R, axis = 1)
#     Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)
    
#     from scipy.sparse.linalg import svds
#     U, sigma, Vt = svds(Ratings_demeaned, k = 50)
#     sigma = np.diag(sigma)
#     all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
#     preds2 = pd.DataFrame(all_user_predicted_ratings, columns = dataset_2.columns)
#     preds2.to_csv('predicted2.csv')
#     return

In [16]:
# svd_new_user_rating(User_id, rating_data_nested_list)

In [17]:
preds.head()

,Unnamed: 0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,300.0,301.0,302.0,303.0,304.0,305.0,306.0,307.0,308.0,309.0
0,0,-0.048949,0.002101,0.011330,-0.077076,0.020332,-0.028672,-0.098129,-0.171783,-0.124071,...,-0.014799,0.060264,0.024905,0.073726,0.022573,-0.011967,0.087599,0.014540,0.252929,0.022718
1,1,-0.186800,-0.007912,-0.042540,-0.898963,-0.084581,-0.028339,-0.150486,-0.101445,-0.115920,...,0.062862,0.079658,0.050724,0.103593,0.065283,-0.166912,-0.039653,0.047565,0.030345,0.012797
2,2,-0.032185,0.019317,-0.108391,-0.063803,-0.080629,-0.027423,-0.091557,-0.053898,-0.083616,...,-0.020945,0.053987,0.057926,0.040231,0.067101,-0.134565,0.029937,0.059610,0.050380,-0.049891
3,3,-0.350374,-0.172312,-0.078371,-0.109807,-0.039780,-0.024423,-0.225578,-0.011290,-0.251320,...,0.175080,0.113178,0.057976,0.132643,0.135032,-0.304662,0.056973,0.082878,0.155382,-0.023058
4,4,-0.024147,0.001343,-0.230989,0.056057,-0.082824,-0.038933,-0.069143,6.120407,-0.082034,...,0.095883,0.056805,0.067464,0.031411,0.040180,-0.135914,0.044329,0.039137,0.075536,-0.045251


In [18]:
preds.rename(columns = {"Unnamed: 0":"Food_ID"}, inplace=True)
preds.reset_index(drop=True,inplace=True)
preds.set_index('Food_ID', inplace=True)
preds.head()

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,300.0,301.0,302.0,303.0,304.0,305.0,306.0,307.0,308.0,309.0
Food_ID,,,,,,,,,,,,,,,,,,,,,
0,-0.048949,0.002101,0.011330,-0.077076,0.020332,-0.028672,-0.098129,-0.171783,-0.124071,-0.043170,...,-0.014799,0.060264,0.024905,0.073726,0.022573,-0.011967,0.087599,0.014540,0.252929,0.022718
1,-0.186800,-0.007912,-0.042540,-0.898963,-0.084581,-0.028339,-0.150486,-0.101445,-0.115920,-0.108364,...,0.062862,0.079658,0.050724,0.103593,0.065283,-0.166912,-0.039653,0.047565,0.030345,0.012797
2,-0.032185,0.019317,-0.108391,-0.063803,-0.080629,-0.027423,-0.091557,-0.053898,-0.083616,-0.091422,...,-0.020945,0.053987,0.057926,0.040231,0.067101,-0.134565,0.029937,0.059610,0.050380,-0.049891
3,-0.350374,-0.172312,-0.078371,-0.109807,-0.039780,-0.024423,-0.225578,-0.011290,-0.251320,-0.117247,...,0.175080,0.113178,0.057976,0.132643,0.135032,-0.304662,0.056973,0.082878,0.155382,-0.023058
4,-0.024147,0.001343,-0.230989,0.056057,-0.082824,-0.038933,-0.069143,6.120407,-0.082034,-0.247019,...,0.095883,0.056805,0.067464,0.031411,0.040180,-0.135914,0.044329,0.039137,0.075536,-0.045251


In [19]:
preds.columns = preds.columns.map(float)

In [20]:
preds2 = pd.read_csv("predicted2.csv")
preds2.head()

,Unnamed: 0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,300.0,301.0,302.0,303.0,304.0,305.0,306.0,307.0,308.0,309.0
0,0,-0.030261,0.003891,-0.002496,-0.128477,-0.024623,-0.011727,-0.073129,-0.194716,-0.138892,...,0.008856,0.049231,0.017807,0.059837,0.024753,-0.020942,0.038325,0.020649,0.222876,0.001379
1,1,-0.221703,-0.011567,-0.043541,-0.993206,-0.006418,-0.032683,-0.139965,-0.138739,-0.087761,...,0.062705,0.082771,0.053308,0.099017,0.068125,-0.190732,-0.033267,0.045541,0.016130,0.004366
2,2,-0.017677,0.019301,-0.098380,0.020810,-0.073856,-0.029277,-0.078654,-0.049702,-0.080726,...,-0.037525,0.052972,0.054384,0.035905,0.065094,-0.128914,0.030027,0.059621,0.053643,-0.046554
3,3,-0.354177,-0.166070,-0.089160,-0.218633,0.013617,-0.033042,-0.216886,-0.044490,-0.233004,...,0.187209,0.116674,0.063442,0.132763,0.141169,-0.329733,0.071333,0.086467,0.150041,-0.036695
4,4,-0.023677,0.005486,-0.216620,0.132789,-0.081113,-0.040259,-0.065426,6.143563,-0.082478,...,0.075485,0.058498,0.065120,0.028668,0.039074,-0.134465,0.046969,0.038601,0.084188,-0.037846


In [21]:
preds2.rename(columns = {"Unnamed: 0":"Food_ID"}, inplace=True)
preds2.reset_index(drop=True,inplace=True)
preds2.set_index('Food_ID', inplace=True)
preds2.head()

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,300.0,301.0,302.0,303.0,304.0,305.0,306.0,307.0,308.0,309.0
Food_ID,,,,,,,,,,,,,,,,,,,,,
0,-0.030261,0.003891,-0.002496,-0.128477,-0.024623,-0.011727,-0.073129,-0.194716,-0.138892,-0.030910,...,0.008856,0.049231,0.017807,0.059837,0.024753,-0.020942,0.038325,0.020649,0.222876,0.001379
1,-0.221703,-0.011567,-0.043541,-0.993206,-0.006418,-0.032683,-0.139965,-0.138739,-0.087761,-0.162531,...,0.062705,0.082771,0.053308,0.099017,0.068125,-0.190732,-0.033267,0.045541,0.016130,0.004366
2,-0.017677,0.019301,-0.098380,0.020810,-0.073856,-0.029277,-0.078654,-0.049702,-0.080726,-0.091877,...,-0.037525,0.052972,0.054384,0.035905,0.065094,-0.128914,0.030027,0.059621,0.053643,-0.046554
3,-0.354177,-0.166070,-0.089160,-0.218633,0.013617,-0.033042,-0.216886,-0.044490,-0.233004,-0.153389,...,0.187209,0.116674,0.063442,0.132763,0.141169,-0.329733,0.071333,0.086467,0.150041,-0.036695
4,-0.023677,0.005486,-0.216620,0.132789,-0.081113,-0.040259,-0.065426,6.143563,-0.082478,-0.253066,...,0.075485,0.058498,0.065120,0.028668,0.039074,-0.134465,0.046969,0.038601,0.084188,-0.037846


In [22]:
preds2.columns = preds2.columns.map(float)

In [23]:
preds.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
            85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
           dtype='int64', name='Food_ID')

In [24]:
preds2.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
             91,  92,  93,  94,  95,  96,  97,  98,  99, 100],
           dtype='int64', name='Food_ID', length=101)

In [45]:
def svd_food_recommendation(predictions, userID, food, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    predictions.index = predictions.index.map(str)
    uu = pd.read_csv("Id.csv")
    uu = uu.astype({"originalId": str})
    fil = uu['originalId'].isin([userID])
    user_row_number = uu.loc[fil]['userId']
    user_row_number = str(int(float(user_row_number))-1)
    print(str(float(user_row_number)+1))
#     user_row_number = str(int(float(userID))-1)
    select = predictions.loc[predictions.index==user_row_number]
    sorted_user_predictions = select.iloc[0,:].sort_values(ascending=False)
    # Get the user's data and merge in the food information.
    user_data = original_ratings[original_ratings.User_ID.map(str) == str(float(user_row_number)+1)]
    user_full = (user_data.merge(food, how = 'left', left_on = 'Food_ID', right_on = 'Food_ID').
                     sort_values(['Rating'], ascending=False)
                 )
    
    sorted_user_predictions_df = pd.DataFrame(sorted_user_predictions)
    sorted_user_predictions_df.index.name = "Food_ID"
#     sorted_user_predictions_df['Food_ID'] = sorted_user_predictions_df.index
    sorted_user_predictions_df = sorted_user_predictions_df.reset_index()
    display(user_data)
    display(sorted_user_predictions_df)
#     print(sorted_user_predictions_df.index)
    
#     Recommend the highest predicted rating foods that the user hasn't tasted yet.
    recommendations = (food[~food['Food_ID'].isin(user_full['Food_ID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'Food_ID',
               right_on = 'Food_ID').
         rename(columns = {str(user_row_number): 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

# svd_food_recommendation(preds, 4, food, ratings, 10)
#data = knn_food_recommendation("chicken biryani")
#print(data)
ratings2 = pd.read_csv("ratings2.csv")
already_rated, predictions = svd_food_recommendation(preds2, '2345.0', food, ratings2, 10)
predictions

101.0


,User_ID,Food_ID,Rating
510,101.0,4.0,6.00
511,101.0,35.0,4.00
512,101.0,44.0,7.25


,Food_ID,100
0,44.0,6.696059
1,4.0,2.669542
2,187.0,1.698798
3,126.0,1.516866
4,35.0,1.404996
...,...,...
304,50.0,-0.351239
305,49.0,-0.369198
306,28.0,-0.380033
307,93.0,-0.518425


,Food_ID,Name,C_Type,Veg_Non,Describe
183,187,tricolour pizza,Italian,veg,"pizza base , pizza sauce, mozzarella cheese, b..."
122,126,andhra crab meat masala,Indian,non-veg,"processed crab meat, refined oil, curry leaves..."
71,75,detox haldi tea,Beverage,veg,"haldi, ginger, black pepper, honey, water"
96,100,spicy chicken curry,Indian,non-veg,"oil, ghee, onion paste, garlic paste, ginger p..."
66,70,shepherds salad (tamatar-kheera salaad),Healthy Food,veg,"1 cucumber peeled and chopped, onion, tomato, ..."
37,40,corn and raw mango salad,Healthy Food,veg,"Corn kernels, onions, green onions, paprika, r..."
128,132,coffee marinated mutton chops,Thai,non-veg,"mutton chops, espresso, honey, balsamic vinega..."
65,69,banana and maple ice lollies,Dessert,veg,"Banana, Greek yogurt, heavy cream, pure maple ..."
264,268,baked wild berry cheesecake,Dessert,veg,"butter, digestive biscuits, berries, cream che..."
246,250,jerk chicken,Indian,non-veg,"chicken legs, lime (halved), jerk seasoning po..."


In [33]:
already_rated.head()

,User_ID,Rating,Food_ID,Name,C_Type,Veg_Non,Describe


In [27]:
# preds.columns = preds.columns.map(float)

In [28]:
# list(preds.columns)

In [29]:
# preds.head(20).index.tolist()

In [30]:
# preds.index = preds.index.map(str)
# # preds.index
# select = preds.loc[preds.index == str(int(float('1.0'))-1)]
# series1 = select.iloc[0,:].sort_values(ascending=False)
# series1
# # select

In [31]:
# preds.iloc[0].sort_values(ascending=False)